In [1]:
import numpy as np
import pandas as pd

## 중분류 별 가중치 (정책자료 크롤링 - 지표 별 언급 빈도 분석)

In [34]:
df.columns

Index(['대분류', '중분류', '소분류', '중요단어', '여가부_2024청소년정책1권', '여가부_2024청소년정책2권',
       '여가부_청소년매체및유해환경실태조사', '교육부_2024학생건강...추진방향', '교육부_2023학생건강...추진방향',
       '질병관리청_청소년건강행태...발표자료', '[개인+외부] 집계 A - 전체'],
      dtype='object')

In [36]:
df = pd.read_csv('./data/weight_crawling_V.1.csv')

# 중분류를 기준으로 그룹화하고, 각 그룹에 대해 해당 컬럼들의 값을 더하고 평균을 구함
grouped = df.groupby('중분류')['[개인+외부] 집계 A - 전체'].mean()

중분류
건강교육        48.00000
공동체/지역사회    124.87500
신체           66.34375
심리           52.25000
위험요인        105.50000
Name: [개인+외부] 집계 A - 전체, dtype: float64

In [93]:
# 각 중분류에 대한 평균 값
health_education = 48.00000
community = 124.87500
physical = 66.34375
mental = 52.25000
risk_factor = 105.50000

# 대분류 별 중분류에 대한 값의 합
total_physical_mental = physical + mental
total_ext_factor = health_education + community + risk_factor

# 가중치 계산
weight_physical = round((physical / total_physical_mental), 4)
weight_mental = round((mental / total_physical_mental), 4)
weight_health_edu = round((health_education / total_ext_factor), 4)
weight_community = round((community / total_ext_factor), 4)
weight_risk_fac = round((risk_factor / total_ext_factor), 4)

# 결과 출력
print("개인건강")
print("신체 중분류의 가중치:", weight_physical)
print("심리 중분류의 가중치:", weight_mental)
print("="*20)
print("외부요인")
print("공동체/지역사회 중분류의 가중치:", weight_community)
print("위험요인 중분류의 가중치:", weight_risk_fac)
print("건강교육 중분류의 가중치:", weight_health_edu)

개인건강
신체 중분류의 가중치: 0.5594
심리 중분류의 가중치: 0.4406
외부요인
공동체/지역사회 중분류의 가중치: 0.4486
위험요인 중분류의 가중치: 0.379
건강교육 중분류의 가중치: 0.1724


## 소분류 별 가중치 - 전문가 설문 결과 (역수 부여)

In [9]:
# Min-Max 정규화 함수
def min_max_normalize(arr):
    min_val = np.min(arr)
    max_val = np.max(arr)
    return (arr - min_val) / (max_val - min_val)

# 가중치 계산 함수
def calculate_weights(priority_values):
    weights = {}
    for category, values in priority_values.items():
        normalized_values = min_max_normalize(values)
        weights[category] = normalized_values / np.sum(normalized_values)
    return weights


In [10]:
# CSV 파일을 읽어들임
# df = pd.read_csv('./지표 및 가중치 설정 - 가중치_지표별 설문_전문가.csv')

# 누락지표 배제_v.2
df = pd.read_csv('./data/지표 및 가중치 설정 - 가중치_지표별 설문_전문가_v.2.csv')

# 중복된 열 이름을 제거하고 중분류와 소분류, 평균 열만 선택
df_subset = df[['중분류', '소분류', '평균']]

# 중분류를 기준으로 그룹화하고 각 그룹에 대해 소분류를 리스트로 모음
grouped = df_subset.groupby('중분류')['소분류'].apply(list)


# 각 중분류마다 소분류들끼리 비교하여 가중치 계산
weights = {}
for category, subcategories in grouped.items():
    # 중복된 소분류를 제거하고 각 소분류별 우선순위를 리스트로 모음
    priorities = df_subset[df_subset['소분류'].isin(subcategories)]['평균'].tolist()
    
    # 우선순위 역수를 취한 후 가중치 계산 (최소 가중치를 0.01로 설정하여 적용)
    min_weight = 0.01
    inverted_priorities = 1 / np.array(priorities)
    normalized_values = min_max_normalize(inverted_priorities)
    weights[category] = (normalized_values + min_weight) / (np.sum(normalized_values) + min_weight * len(normalized_values))

# 결과 출력
for category, subcategories in grouped.items():
    print(f"{category}의 소분류별 가중치:")
    for subcategory, weight in zip(subcategories, weights[category]):
        print(f"{subcategory}: {weight:.3f}")
    print()

건강교육의 소분류별 가중치:
성교육: 0.338
학교폭력 교육: 0.271
청소년 매체 유해교육: 0.204
안전 교육: 0.155
영양/식습관 교육: 0.029
개인 위생 교육: 0.003

공동체/지역사회의 소분류별 가중치:
가족(보호자) 상호작용: 0.541
교사 상호작용: 0.118
친구 상호작용: 0.305
청소년 기관/센터 인지: 0.031
청소년 기관/센터 경험: 0.005

신체의 소분류별 가중치:
신체활동: 0.470
비만: 0.123
흡연: 0.136
음주: 0.083
식생활: 0.136
구강건강: 0.005
개인위생: 0.035
아토피/천식: 0.012

심리의 소분류별 가중치:
수면: 0.060
우울: 0.284
자살생각여부: 0.284
스트레스: 0.230
외로움: 0.040
범불안장애: 0.003
행복감: 0.099

위험요인의 소분류별 가중치:
학교폭력: 0.660
성폭력/성범죄: 0.250
유해업소: 0.084
사행/대출: 0.007



#### 일부 카테고리 이름 정제

In [74]:
import re

# CSV 파일 경로
file_path = './data/지표 및 가중치 설정 - 가중치_지표별 설문_전문가.csv'

# CSV 파일을 DataFrame으로 읽어옴
df = pd.read_csv(file_path)

# '건강교육'과 유사한 카테고리를 찾아 이름을 수정
for idx, row in df.iterrows():
    category = row['중분류']
    if re.search(r'건강교육', category):
        df.at[idx, '중분류'] = '건강교육'

# 수정된 DataFrame을 CSV 파일로 저장
df.to_csv(file_path, index=False)

In [14]:
import re

# 각 가중치를 소수점 네 자리까지 반올림하여 표시하는 함수
def round_weights(weights):
    rounded_weights = {}
    for category, sub_weights in weights.items():
        rounded_sub_weights = {subcategory: round(weight, 4) for subcategory, weight in sub_weights.items()}
        rounded_weights[category] = rounded_sub_weights
    return rounded_weights

# 각 중분류마다 소분류들끼리 비교하여 가중치 계산하고 결과를 딕셔너리에 저장
weights_physical_sub = {}
weights_mental_sub = {}
weights_health_education_sub = {}
weights_community_sub = {}
weights_risk_sub = {}

for category, subcategories in grouped.items():
    priorities = df_subset[df_subset['소분류'].isin(subcategories)]['평균'].tolist()
    min_weight = 0.01
    inverted_priorities = 1 / np.array(priorities)
    normalized_values = min_max_normalize(inverted_priorities)
    total_normalized_values = np.sum(normalized_values)
    total_weight = total_normalized_values + min_weight * len(normalized_values)
    normalized_weights = (normalized_values + min_weight) / total_weight

    # 카테고리 이름에 '건강교육'이 포함된 경우 건강교육 카테고리로 저장
    if re.search(r'건강교육', category):
        weights_health_education_sub[category] = dict(zip(subcategories, normalized_weights))
    elif category == '공동체/지역사회':
        weights_community_sub[category] = dict(zip(subcategories, normalized_weights))
    elif category == '신체':
        weights_physical_sub[category] = dict(zip(subcategories, normalized_weights))
    elif category == '심리':
        weights_mental_sub[category] = dict(zip(subcategories, normalized_weights))
    elif category == '위험요인':
        weights_risk_sub[category] = dict(zip(subcategories, normalized_weights))


weights_physical_sub = round_weights(weights_physical_sub)
weights_mental_sub = round_weights(weights_mental_sub)
weights_health_education_sub = round_weights(weights_health_education_sub)
weights_community_sub = round_weights(weights_community_sub)
weights_risk_sub = round_weights(weights_risk_sub)

import pandas as pd

# 결과를 DataFrame으로 변환
adjusted_weights_health_edu_df = pd.DataFrame(weights_health_education_sub).T
adjusted_weights_community_df = pd.DataFrame(weights_community_sub).T
adjusted_weights_physical_df = pd.DataFrame(weights_physical_sub).T
adjusted_weights_mental_df = pd.DataFrame(weights_mental_sub).T
adjusted_weights_risk_df = pd.DataFrame(weights_risk_sub).T

# 모든 DataFrame을 병합
adjusted_weights_df = pd.concat([adjusted_weights_health_edu_df, 
                                 adjusted_weights_community_df, 
                                 adjusted_weights_physical_df, 
                                 adjusted_weights_mental_df, 
                                 adjusted_weights_risk_df], 
                                keys=['건강교육', '공동체/지역사회', '신체', '심리', '위험요인'])

# DataFrame을 CSV 파일로 저장
adjusted_weights_df.to_csv('adjusted_weights_v1.csv', index_label=['category', 'subcategory'])


# 결과 출력
print("건강교육의 소분류별 가중치:")
print(weights_health_education_sub)
print()

print("공동체/지역사회의 소분류별 가중치:")
print(weights_community_sub)
print()

print("신체의 소분류별 가중치:")
print(weights_physical_sub)
print()

print("심리의 소분류별 가중치:")
print(weights_mental_sub)
print()

print("위험요인의 소분류별 가중치:")
print(weights_risk_sub)


건강교육의 소분류별 가중치:
{'건강교육': {'성교육': 0.3379, '학교폭력 교육': 0.2707, '청소년 매체 유해교육': 0.2043, '안전 교육': 0.155, '영양/식습관 교육': 0.0287, '개인 위생 교육': 0.0033}}

공동체/지역사회의 소분류별 가중치:
{'공동체/지역사회': {'가족(보호자) 상호작용': 0.5407, '교사 상호작용': 0.1178, '친구 상호작용': 0.305, '청소년 기관/센터 인지': 0.0311, '청소년 기관/센터 경험': 0.0054}}

신체의 소분류별 가중치:
{'신체': {'신체활동': 0.4701, '비만': 0.1233, '흡연': 0.1358, '음주': 0.0827, '식생활': 0.1358, '구강건강': 0.0047, '개인위생': 0.0354, '아토피/천식': 0.0123}}

심리의 소분류별 가중치:
{'심리': {'수면': 0.0598, '우울': 0.2843, '자살생각여부': 0.2843, '스트레스': 0.2299, '외로움': 0.0402, '범불안장애': 0.0028, '행복감': 0.0987}}

위험요인의 소분류별 가중치:
{'위험요인': {'학교폭력': 0.6597, '성폭력/성범죄': 0.2503, '유해업소': 0.0835, '사행/대출': 0.0065}}


### 복합 가중치 계산모델 (중분류 소속 소분류 가중치의 합 = 중분류의 가중치)

In [12]:
# 개인건강 중분류의 가중치    
    
weight_physical = 0.5594
weight_mental = 0.4406

# 외부요인 중분류의 가중치
weight_community = 0.4486
weight_risk_fac = 0.379
weight_health_edu = 0.1724

# 각 중분류의 소분류 가중치 합산
total_weights_health_education_sub = sum(weights_health_education_sub['건강교육'].values())
total_weights_community_sub = sum(weights_community_sub['공동체/지역사회'].values())
total_weights_physical_sub = sum(weights_physical_sub['신체'].values())
total_weights_mental_sub = sum(weights_mental_sub['심리'].values())
total_weights_risk_sub = sum(weights_risk_sub['위험요인'].values())

# 각 중분류의 가중치 계산
adjusted_weight_health_edu = {subcategory: round(weight * weight_health_edu / total_weights_health_education_sub, 4) for subcategory, weight in weights_health_education_sub['건강교육'].items()}
adjusted_weight_community = {subcategory: round(weight * weight_community / total_weights_community_sub, 4) for subcategory, weight in weights_community_sub['공동체/지역사회'].items()}
adjusted_weight_physical = {subcategory: round(weight * weight_physical / total_weights_physical_sub, 4) for subcategory, weight in weights_physical_sub['신체'].items()}
adjusted_weight_mental = {subcategory: round(weight * weight_mental / total_weights_mental_sub, 4) for subcategory, weight in weights_mental_sub['심리'].items()}
adjusted_weight_risk = {subcategory: round(weight * weight_risk_fac / total_weights_risk_sub, 4) for subcategory, weight in weights_risk_sub['위험요인'].items()}


# 결과를 DataFrame으로 변환
adjusted_weights_df = pd.DataFrame({
    '건강교육': adjusted_weight_health_edu,
    '공동체/지역사회': adjusted_weight_community,
    '신체': adjusted_weight_physical,
    '정신': adjusted_weight_mental,
    '위험요인': adjusted_weight_risk
})

# DataFrame을 CSV 파일로 저장
adjusted_weights_df.to_csv('adjusted_weights.csv', index_label='subcategory')


# 결과 출력
print("건강교육 중분류의 소분류별 가중치 (중분류 반영):")
print(adjusted_weight_health_edu)
print()

print("공동체/지역사회 중분류의 소분류별 가중치 (중분류 반영):")
print(adjusted_weight_community)
print()

print("신체 중분류의 소분류별 가중치 (중분류 반영):")
print(adjusted_weight_physical)
print()

print("정신 중분류의 소분류별 가중치 (중분류 반영):")
print(adjusted_weight_mental)
print()

print("위험요인 중분류의 소분류별 가중치 (중분류 반영):")
print(adjusted_weight_risk)


건강교육 중분류의 소분류별 가중치 (중분류 반영):
{'성교육': 0.0583, '학교폭력 교육': 0.0467, '청소년 매체 유해교육': 0.0352, '안전 교육': 0.0267, '영양/식습관 교육': 0.0049, '개인 위생 교육': 0.0006}

공동체/지역사회 중분류의 소분류별 가중치 (중분류 반영):
{'가족(보호자) 상호작용': 0.2426, '교사 상호작용': 0.0528, '친구 상호작용': 0.1368, '청소년 기관/센터 인지': 0.014, '청소년 기관/센터 경험': 0.0024}

신체 중분류의 소분류별 가중치 (중분류 반영):
{'신체활동': 0.2629, '비만': 0.069, '흡연': 0.076, '음주': 0.0463, '식생활': 0.076, '구강건강': 0.0026, '개인위생': 0.0198, '아토피/천식': 0.0069}

정신 중분류의 소분류별 가중치 (중분류 반영):
{'수면': 0.0263, '우울': 0.1253, '자살생각여부': 0.1253, '스트레스': 0.1013, '외로움': 0.0177, '범불안장애': 0.0012, '행복감': 0.0435}

위험요인 중분류의 소분류별 가중치 (중분류 반영):
{'학교폭력': 0.25, '성폭력/성범죄': 0.0949, '유해업소': 0.0316, '사행/대출': 0.0025}


In [13]:
# 신체건강 중분류의 가중치 합과 비교
total_adjusted_weight_physical_sub = sum(adjusted_weight_physical.values())
print("개인건강 중분류의 가중치 합:", round(total_adjusted_weight_physical_sub, 4))
print("개인건강 중분류의 가중치와 일치 여부:", round(total_adjusted_weight_physical_sub, 4) == round(weight_physical, 4))
print()

# 정신 중분류의 가중치 합과 비교
total_adjusted_weight_mental_sub = sum(adjusted_weight_mental.values())
print("정신 중분류의 가중치 합:", total_adjusted_weight_mental_sub)
print("정신 중분류의 가중치와 일치 여부:", total_adjusted_weight_mental_sub == weight_mental)
print()

# 공동체/지역사회 중분류의 가중치 합과 비교
total_adjusted_weight_community_sub = sum(adjusted_weight_community.values())
print("공동체/지역사회 중분류의 가중치 합:", total_adjusted_weight_community_sub)
print("공동체/지역사회 중분류의 가중치와 일치 여부:", total_adjusted_weight_community_sub == weight_community)
print()

# 위험요인 중분류의 가중치 합과 비교
total_adjusted_weight_risk_sub = sum(adjusted_weight_risk.values())
print("위험요인 중분류의 가중치 합:", total_adjusted_weight_risk_sub)
print("위험요인 중분류의 가중치와 일치 여부:", total_adjusted_weight_risk_sub == weight_risk_fac)
print()

# 건강교육 중분류의 가중치 합과 비교
total_adjusted_weight_health_edu_sub = sum(adjusted_weight_health_edu.values())
print("건강교육 중분류의 가중치 합:", total_adjusted_weight_health_edu_sub)
print("건강교육 중분류의 가중치와 일치 여부:", total_adjusted_weight_health_edu_sub == weight_health_edu)


개인건강 중분류의 가중치 합: 0.5595
개인건강 중분류의 가중치와 일치 여부: False

정신 중분류의 가중치 합: 0.44059999999999994
정신 중분류의 가중치와 일치 여부: False

공동체/지역사회 중분류의 가중치 합: 0.44860000000000005
공동체/지역사회 중분류의 가중치와 일치 여부: False

위험요인 중분류의 가중치 합: 0.379
위험요인 중분류의 가중치와 일치 여부: True

건강교육 중분류의 가중치 합: 0.17239999999999997
건강교육 중분류의 가중치와 일치 여부: False
